# Régression Logistique avec une mentalité de Réseau de Neurones

Bienvenue dans votre premier devoir de programmation (obligatoire) ! Vous allez construire un classifieur par régression logistique pour reconnaître des chats. Ce devoir vous guidera étape par étape pour réaliser cela avec une "mentalité" de réseau de neurones, ce qui affinera également vos intuitions sur le Deep Learning.

**Instructions :**
- N'utilisez pas de boucles (for/while) dans votre code, sauf si les instructions vous le demandent explicitement.
- Utilisez `np.dot(X,Y)` pour calculer les produits scalaires.

**Vous apprendrez à :**
- Construire l'architecture générale d'un algorithme d'apprentissage, y compris :
    - L'initialisation des paramètres
    - Le calcul de la fonction de coût et de son gradient
    - L'utilisation d'un algorithme d'optimisation (descente de gradient) 
- Rassembler les trois fonctions ci-dessus dans une fonction modèle principale, dans le bon ordre.


Avant de soumettre votre devoir, assurez-vous de ne pas faire ce qui suit :

1. Vous n'avez pas ajouté d'instructions `print` _supplémentaires_ dans le devoir.
2. Vous n'avez pas ajouté de cellules de code _supplémentaires_.
3. Vous n'avez pas modifié les paramètres des fonctions.
4. Vous n'utilisez pas de variables globales à l'intérieur de vos exercices notés.
5. Vous ne modifiez pas le code du devoir là où ce n'est pas requis.



## Table des Matières
- [1 - Packages](#1)
- [2 - Vue d'ensemble du problème](#2)
    - [Exercice 1](#ex-1)
    - [Exercice 2](#ex-2)
- [3 - Architecture générale de l'algorithme d'apprentissage](#3)
- [4 - Construction des parties de notre algorithme](#4)
    - [4.1 - Fonctions auxiliaires](#4-1)
        - [Exercice 3 - sigmoid](#ex-3)
    - [4.2 - Initialisation des paramètres](#4-2)
        - [Exercice 4 - initialize_with_zeros](#ex-4)
    - [4.3 - Propagation avant et arrière](#4-3)
        - [Exercice 5 - propagate](#ex-5)
    - [4.4 - Optimisation](#4-4)
        - [Exercice 6 - optimize](#ex-6)
        - [Exercice 7 - predict](#ex-7)
- [5 - Fusionner toutes les fonctions dans un modèle](#5)
    - [Exercice 8 - model](#ex-8)
- [6 - Analyse approfondie (optionnel/non noté)](#6)
- [7 - Test avec votre propre image (optionnel/non noté)](#7)

<a name='1'></a>
## 1 - Packages ##

Tout d'abord, exécutons la cellule ci-dessous pour importer tous les packages dont vous aurez besoin lors de ce devoir. 
- [numpy](https://numpy.org/doc/1.20/) est le package fondamental pour le calcul scientifique avec Python.
- [h5py](http://www.h5py.org) est un package courant pour interagir avec un ensemble de données stocké dans un fichier H5.
- [matplotlib](http://matplotlib.org) est une bibliothèque célèbre pour tracer des graphiques en Python.
- [PIL](https://pillow.readthedocs.io/en/stable/) et [scipy](https://www.scipy.org/) sont utilisés ici pour tester votre modèle avec votre propre image à la fin.

### v1.2

In [ ]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset
from public_tests import *


<a name='2'></a>
## 2 - Vue d'ensemble du problème ##

**Énoncé du problème** : On vous donne un jeu de données ("data.h5") contenant :
    - un ensemble d'entraînement de m_train images étiquetées comme chat (y=1) ou non-chat (y=0)
    - un ensemble de test de m_test images étiquetées comme chat ou non-chat
    - chaque image est de forme (num_px, num_px, 3) où 3 correspond aux 3 canaux (RGB). Ainsi, chaque image est carrée (hauteur = num_px) et (largeur = num_px).

Vous allez construire un algorithme simple de reconnaissance d'image capable de classer correctement les images comme chat ou non-chat.

Familiarisons-nous davantage avec le jeu de données. Chargez les données en exécutant le code suivant.

In [5]:
# Chargement des données (chat/non-chat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

Nous avons ajouté "_orig" à la fin des jeux de données d'images (train et test) car nous allons les prétraiter. Après le prétraitement, nous obtiendrons train_set_x et test_set_x (les étiquettes train_set_y et test_set_y n'ont besoin d'aucun prétraitement).

Chaque ligne de votre train_set_x_orig et test_set_x_orig est un tableau représentant une image. Vous pouvez visualiser un exemple en exécutant le code suivant. N'hésitez pas à changer la valeur de `index` et à relancer pour voir d'autres images.

In [8]:
# Exemple d'une image
index = 25
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:, index]) + ", c'est une image de '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "'.")

De nombreux bugs logiciels en Deep Learning proviennent de dimensions de matrices/vecteurs qui ne correspondent pas. Si vous parvenez à maintenir vos dimensions de matrice/vecteur correctes, vous éliminerez une grande partie des bugs. 

<a name='ex-1'></a>
### Exercice 1
Trouvez les valeurs pour :
    - m_train (nombre d'exemples d'entraînement)
    - m_test (nombre d'exemples de test)
    - num_px (= hauteur = largeur d'une image d'entraînement)
Rappelez-vous que `train_set_x_orig` est un tableau numpy de forme (m_train, num_px, num_px, 3). Par exemple, vous pouvez accéder à `m_train` en écrivant `train_set_x_orig.shape[0]`.

In [ ]:
#(≈ 3 lignes de code)
# m_train = 
# m_test = 
# num_px = 
# VOTRE CODE COMMENCE ICI


# VOTRE CODE SE TERMINE ICI

print ("Nombre d'exemples d'entraînement : m_train = " + str(m_train))
print ("Nombre d'exemples de test : m_test = " + str(m_test))
print ("Hauteur/Largeur de chaque image : num_px = " + str(num_px))
print ("Chaque image est de taille : (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("forme de train_set_x : " + str(train_set_x_orig.shape))
print ("forme de train_set_y : " + str(train_set_y.shape))
print ("forme de test_set_x : " + str(test_set_x_orig.shape))
print ("forme de test_set_y : " + str(test_set_y.shape))

**Résultat attendu pour m_train, m_test et num_px** : 
<table style="width:15%">
  <tr>
    <td> m_train </td>
    <td> 209 </td> 
  </tr>
  
  <tr>
    <td>m_test</td>
    <td> 50 </td> 
  </tr>
  
  <tr>
    <td>num_px</td>
    <td> 64 </td> 
  </tr>
  
</table>


Pour plus de commodité, vous devez maintenant redimensionner les images de forme (num_px, num_px, 3) en un tableau numpy de forme (num_px $*$ num_px $*$ 3, 1). Après cela, notre jeu de données d'entraînement (et de test) sera un tableau numpy où chaque colonne représente une image aplatie. Il devrait y avoir m_train (respectivement m_test) colonnes.

<a name='ex-2'></a>
### Exercice 2
Redimensionnez les jeux de données d'entraînement et de test de sorte que les images de taille (num_px, num_px, 3) soient aplaties en vecteurs uniques de forme (num\_px $*$ num\_px $*$ 3, 1).

Une astuce lorsque vous voulez aplatir une matrice X de forme (a,b,c,d) en une matrice X_flatten de forme (b$*$c$*$d, a) est d'utiliser : 
```python
X_flatten = X.reshape(X.shape[0], -1).T      # X.T est la transposée de X
```

In [ ]:
# Redimensionner les exemples d'entraînement et de test
#(≈ 2 lignes de code)
# train_set_x_flatten = ...
# test_set_x_flatten = ...
# VOTRE CODE COMMENCE ICI


# VOTRE CODE SE TERMINE ICI

# Vérification que les 10 premiers pixels de la deuxième image sont au bon endroit
assert np.alltrue(train_set_x_flatten[0:10, 1] == [196, 192, 190, 193, 186, 182, 188, 179, 174, 213]), "Mauvaise solution. Utilisez (X.shape[0], -1).T."
assert np.alltrue(test_set_x_flatten[0:10, 1] == [115, 110, 111, 137, 129, 129, 155, 146, 145, 159]), "Mauvaise solution. Utilisez (X.shape[0], -1).T."

print ("forme de train_set_x_flatten : " + str(train_set_x_flatten.shape))
print ("forme de train_set_y : " + str(train_set_y.shape))
print ("forme de test_set_x_flatten : " + str(test_set_x_flatten.shape))
print ("forme de test_set_y : " + str(test_set_y.shape))

**Résultat attendu**: 

<table style="width:35%">
  <tr>
    <td>train_set_x_flatten shape</td>
    <td> (12288, 209)</td> 
  </tr>
  <tr>
    <td>train_set_y shape</td>
    <td>(1, 209)</td> 
  </tr>
  <tr>
    <td>test_set_x_flatten shape</td>
    <td>(12288, 50)</td> 
  </tr>
  <tr>
    <td>test_set_y shape</td>
    <td>(1, 50)</td> 
  </tr>
</table>

Pour représenter des images couleur, les canaux rouge, vert et bleu (RGB) doivent être spécifiés pour chaque pixel, et donc la valeur du pixel est en réalité un vecteur de trois nombres allant de 0 à 255.

Une étape de prétraitement courante en apprentissage automatique consiste à centrer et standardiser votre jeu de données, ce qui signifie que vous soustrayez la moyenne de tout le tableau numpy à chaque exemple, puis divisez chaque exemple par l'écart type de tout le tableau numpy. Mais pour les jeux de données d'images, il est plus simple, plus pratique et presque aussi efficace de simplement diviser chaque ligne du jeu de données par 255 (la valeur maximale d'un canal de pixel).

Standardisons notre jeu de données.

In [ ]:
train_set_x = train_set_x_flatten / 255.
test_set_x = test_set_x_flatten / 255.

<font color='blue'>
    
    
**Ce qu'il faut retenir :**

Les étapes communes pour le prétraitement d'un nouveau jeu de données sont :
- Comprendre les dimensions et les formes du problème (m_train, m_test, num_px, ...)
- Redimensionner les jeux de données de telle sorte que chaque exemple soit maintenant un vecteur de taille (num_px \* num_px \* 3, 1)
- "Standardiser" les données

<a name='3'></a>
## 3 - Architecture générale de l'algorithme d'apprentissage ##

Il est temps de concevoir un algorithme simple pour distinguer les images de chats des images sans chats.

Vous allez construire une Régression Logistique, en utilisant une mentalité de Réseau de Neurones. La figure suivante explique pourquoi **la Régression Logistique est en fait un Réseau de Neurones très simple !**

<img src="images/LogReg_kiank.png" style="width:650px;height:400px;">

**Expression mathématique de l'algorithme**:

Pour un exemple $x^{(i)}$:

$$
z^{(i)} = w^T x^{(i)} + b \tag{1}
$$

$$
\hat{y}^{(i)} = a^{(i)} = \sigma\bigl(z^{(i)}\bigr) \tag{2}
$$

$$
\mathcal{L}\bigl(a^{(i)}, y^{(i)}\bigr) = -\, y^{(i)} \log\bigl(a^{(i)}\bigr) - \bigl(1 - y^{(i)}\bigr) \log\bigl(1 - a^{(i)}\bigr) \tag{3}
$$

Le coût est ensuite calculé en sommant sur tous les exemples d'entraînement :

$$
J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}\bigl(a^{(i)}, y^{(i)}\bigr) \tag{6}
$$

<a name='4'></a>
## 4 - Construction des parties de notre algorithme ## 

Les principales étapes pour construire un Réseau de Neurones sont :
1. Définir la structure du modèle (comme le nombre de caractéristiques en entrée) 
2. Initialiser les paramètres du modèle
3. Boucle :
    - Calculer la perte actuelle (propagation avant)
    - Calculer le gradient actuel (propagation arrière)
    - Mettre à jour les paramètres (descente de gradient)

Vous construisez souvent 1-3 séparément et les intégrez dans une seule fonction que nous appelons `model()`.

<a name='4-1'></a>
### 4.1 - Fonctions auxiliaires

<a name='ex-3'></a>
### Exercice 3 - sigmoid
En utilisant votre code des "Bases de Python", implémentez `sigmoid()`. Comme vous l'avez vu dans la figure ci-dessus, vous devez calculer $sigmoid(z) = \frac{1}{1 + e^{-z}}$ pour $z = w^T x + b$ afin de faire des prédictions. Utilisez np.exp().

In [ ]:
# GRADED FUNCTION: sigmoid

def sigmoid(z):
    """
    Calcule la sigmoïde de z

    Arguments :
    z -- Un scalaire ou un tableau numpy de n'importe quelle taille.

    Retour :
    s -- sigmoid(z)
    """

    #(≈ 1 ligne de code)
    # s = ...
    # VOTRE CODE COMMENCE ICI
    
    
    # VOTRE CODE SE TERMINE ICI
    
    return s

In [ ]:
print ("sigmoid([0, 2]) = " + str(sigmoid(np.array([0,2]))))

sigmoid_test(sigmoid)

In [ ]:
x = np.array([0.5, 0, 2.0])
output = sigmoid(x)
print(output)

<a name='4-2'></a>
### 4.2 - Initialisation des paramètres

<a name='ex-4'></a>
### Exercice 4 - initialize_with_zeros
Implémentez l'initialisation des paramètres dans la cellule ci-dessous. Vous devez initialiser w comme un vecteur de zéros. Si vous ne savez pas quelle fonction numpy utiliser, cherchez np.zeros() dans la documentation de la bibliothèque Numpy.

In [ ]:
# GRADED FUNCTION: initialize_with_zeros

def initialize_with_zeros(dim):
    """
    Cette fonction crée un vecteur de zéros de forme (dim, 1) pour w et initialise b à 0.
    
    Argument :
    dim -- taille du vecteur w que nous voulons (ou nombre de paramètres dans ce cas)
    
    Retours :
    w -- vecteur initialisé de forme (dim, 1)
    b -- scalaire initialisé (correspond au biais) de type float
    """
    
    # (≈ 2 lignes de code)
    # w = ...
    # b = ...
    # VOTRE CODE COMMENCE ICI
    
    
    # VOTRE CODE SE TERMINE ICI

    return w, b

In [ ]:
dim = 2
w, b = initialize_with_zeros(dim)

assert type(b) == float
print ("w = " + str(w))
print ("b = " + str(b))

initialize_with_zeros_test_1(initialize_with_zeros)
initialize_with_zeros_test_2(initialize_with_zeros)


<a name='4-3'></a>
### 4.3 - Propagation avant et arrière

Maintenant que vos paramètres sont initialisés, vous pouvez effectuer les étapes de propagation "avant" et "arrière" pour apprendre les paramètres.

<a name='ex-5'></a>
### Exercice 5 - propagate
Implémentez une fonction `propagate()` qui calcule la fonction de coût et son gradient.

**Indices** :

Propagation Avant (Forward Propagation) :
- Vous obtenez X
- Vous calculez $A = \sigma(w^T X + b) = (a^{(1)}, a^{(2)}, ..., a^{(m-1)}, a^{(m)})$
- Vous calculez la fonction de coût : $J = -\frac{1}{m}\sum_{i=1}^{m}(y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)}))$

Voici les deux formules que vous utiliserez : 

$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T\tag{7}$$
$$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})\tag{8}$$

In [ ]:
# GRADED FUNCTION: propagate

def propagate(w, b, X, Y):
    """
    Implémente la fonction de coût et son gradient pour la propagation expliquée ci-dessus

    Arguments :
    w -- poids, un tableau numpy de taille (num_px * num_px * 3, 1)
    b -- biais, un scalaire
    X -- données de taille (num_px * num_px * 3, nombre d'exemples)
    Y -- vecteur des vraies "étiquettes" (contenant 0 si non-chat, 1 si chat) de taille (1, nombre d'exemples)

    Retour :
    grads -- dictionnaire contenant les gradients des poids et du biais
            (dw -- gradient de la perte par rapport à w, donc même forme que w)
            (db -- gradient de la perte par rapport à b, donc même forme que b)
    cost -- coût log-vraisemblance négative pour la régression logistique
    
    Conseils :
    - Écrivez votre code étape par étape pour la propagation. np.log(), np.dot()
    """
    
    m = X.shape[1]
    
    # PROPAGATION AVANT (DE X AU COÛT)
    #(≈ 2 lignes de code)
    # calculez l'activation
    # A = ...
    # calculez le coût en utilisant np.dot pour effectuer la multiplication. 
    # Et n'utilisez pas de boucles pour la somme.
    # cost = ...                                
    # VOTRE CODE COMMENCE ICI
    
    
    # VOTRE CODE SE TERMINE ICI

    # RÉTROPROPAGATION (POUR TROUVER LE GRAD)
    #(≈ 2 lignes de code)
    # dw = ...
    # db = ...
    # VOTRE CODE COMMENCE ICI
    
    
    # VOTRE CODE SE TERMINE ICI
    cost = np.squeeze(np.array(cost))

    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [ ]:
w =  np.array([[1.], [2]])
b = 1.5

# X is using 3 examples, with 2 features each
# Each example is stacked column-wise
X = np.array([[1., -2., -1.], [3., 0.5, -3.2]])
Y = np.array([[1, 1, 0]])
grads, cost = propagate(w, b, X, Y)

assert type(grads["dw"]) == np.ndarray
assert grads["dw"].shape == (2, 1)
assert type(grads["db"]) == np.float64


print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

propagate_test(propagate)

**Expected output**

```
dw = [[ 0.25071532]
 [-0.06604096]]
db = -0.1250040450043965
cost = 0.15900537707692405
```

<a name='4-4'></a>
### 4.4 - Optimisation
- Vous avez initialisé vos paramètres.
- Vous êtes également capable de calculer une fonction de coût et son gradient.
- Maintenant, vous voulez mettre à jour les paramètres en utilisant la descente de gradient.

<a name='ex-6'></a>
### Exercice 6 - optimize
Écrivez la fonction d'optimisation. Le but est d'apprendre $w$ et $b$ en minimisant la fonction de coût $J$. Pour un paramètre $\theta$, la règle de mise à jour est $ \theta = \theta - \alpha \text{ } d\theta$, où $\alpha$ est le taux d'apprentissage.

In [ ]:
# GRADED FUNCTION: optimize

def optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009, print_cost=False):
    """
    Cette fonction optimise w et b en exécutant un algorithme de descente de gradient
    
    Arguments :
    w -- poids, un tableau numpy de taille (num_px * num_px * 3, 1)
    b -- biais, un scalaire
    X -- données de forme (num_px * num_px * 3, nombre d'exemples)
    Y -- vecteur des vraies "étiquettes" (0 si non-chat, 1 si chat), de forme (1, nombre d'exemples)
    num_iterations -- nombre d'itérations de la boucle d'optimisation
    learning_rate -- taux d'apprentissage de la règle de mise à jour de la descente de gradient
    print_cost -- True pour afficher la perte tous les 100 pas
    
    Retours :
    params -- dictionnaire contenant les poids w et le biais b
    grads -- dictionnaire contenant les gradients des poids et du biais par rapport à la fonction de coût
    costs -- liste de tous les coûts calculés pendant l'optimisation, cela sera utilisé pour tracer la courbe d'apprentissage.
    
    Conseils :
    Vous devez fondamentalement écrire deux étapes et itérer à travers elles :
        1) Calculer le coût et le gradient pour les paramètres actuels. Utilisez propagate().
        2) Mettre à jour les paramètres en utilisant la règle de descente de gradient pour w et b.
    """
    
    w = copy.deepcopy(w)
    b = copy.deepcopy(b)
    
    costs = []
    
    for i in range(num_iterations):
        # (≈ 1 ligne de code)
        # Calcul du coût et du gradient 
        # grads, cost = ...
        # VOTRE CODE COMMENCE ICI
        
        
        # VOTRE CODE SE TERMINE ICI
        
        # Récupérer les dérivées depuis grads
        dw = grads["dw"]
        db = grads["db"]
        
        # Règle de mise à jour (≈ 2 lignes de code)
        # w = ...
        # b = ...
        # VOTRE CODE COMMENCE ICI
        
        
        # VOTRE CODE SE TERMINE ICI
        
        # Enregistrer les coûts
        if i % 100 == 0:
            costs.append(cost)
        
            # Afficher le coût toutes les 100 itérations d'entraînement
            if print_cost:
                print ("Coût après l'itération %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [ ]:
params, grads, costs = optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009, print_cost=False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print("Costs = " + str(costs))

optimize_test(optimize)

<a name='ex-7'></a>
### Exercice 7 - predict
La fonction précédente produira les paramètres w et b appris. Nous pouvons utiliser w et b pour prédire les étiquettes pour un jeu de données X. Implémentez la fonction `predict()`. Il y a deux étapes pour calculer les prédictions :

1. Calculer $\hat{Y} = A = \sigma(w^T X + b)$

2. Convertir les entrées de a en 0 (si activation <= 0.5) ou 1 (si activation > 0.5), stocker les prédictions dans un vecteur `Y_prediction`. Si vous le souhaitez, vous pouvez utiliser une instruction `if`/`else` dans une boucle `for` (bien qu'il existe aussi un moyen de vectoriser cela). 

In [ ]:
# GRADED FUNCTION: predict

def predict(w, b, X):
    '''
    Prédire si l'étiquette est 0 ou 1 en utilisant les paramètres de régression logistique appris (w, b)
    
    Arguments :
    w -- poids, un tableau numpy de taille (num_px * num_px * 3, 1)
    b -- biais, un scalaire
    X -- données de taille (num_px * num_px * 3, nombre d'exemples)
    
    Retours :
    Y_prediction -- un tableau numpy (vecteur) contenant toutes les prédictions (0/1) pour les exemples dans X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    # Calculer le vecteur "A" prédisant les probabilités de la présence d'un chat dans l'image
    #(≈ 1 ligne de code)
    # A = ...
    # VOTRE CODE COMMENCE ICI
    
    
    # VOTRE CODE SE TERMINE ICI
    
    for i in range(A.shape[1]):
        
        # Convertir les probabilités A[0,i] en prédictions réelles Y_prediction[0,i]
        #(≈ 4 lignes de code)
        # if A[0, i] > ____ :
        #     Y_prediction[0,i] = 
        # else:
        #     Y_prediction[0,i] = 
        # VOTRE CODE COMMENCE ICI
        
        
        # VOTRE CODE SE TERMINE ICI
    
    return Y_prediction

In [ ]:
w = np.array([[0.1124579], [0.23106775]])
b = -0.3
X = np.array([[1., -1.1, -3.2],[1.2, 2., 0.1]])
print ("predictions = " + str(predict(w, b, X)))

predict_test(predict)

<font color='blue'>
    
**Ce qu'il faut retenir :**
    
Vous avez implémenté plusieurs fonctions qui :
- Initialisent (w,b)
- Optimisent la perte itérativement pour apprendre les paramètres (w,b) :
    - Calcul du coût et de son gradient 
    - Mise à jour des paramètres par descente de gradient
- Utilisent les (w,b) appris pour prédire les étiquettes pour un ensemble d'exemples donné

<a name='5'></a>
## 5 - Fusionner toutes les fonctions dans un modèle ##

Vous allez maintenant voir comment le modèle global est structuré en rassemblant tous les blocs de construction (fonctions implémentées dans les parties précédentes) ensemble, dans le bon ordre.

<a name='ex-8'></a>
### Exercice 8 - model
Implémentez la fonction modèle. Utilisez la notation suivante :
    - Y_prediction_test pour vos prédictions sur l'ensemble de test
    - Y_prediction_train pour vos prédictions sur l'ensemble d'entraînement
    - parameters, grads, costs pour les sorties de optimize()

In [ ]:
# GRADED FUNCTION: model

def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
    """
    Construit le modèle de régression logistique en appelant la fonction que vous avez implémentée précédemment
    
    Arguments :
    X_train -- ensemble d'entraînement représenté par un tableau numpy de forme (num_px * num_px * 3, m_train)
    Y_train -- étiquettes d'entraînement représentées par un tableau numpy (vecteur) de forme (1, m_train)
    X_test -- ensemble de test représenté par un tableau numpy de forme (num_px * num_px * 3, m_test)
    Y_test -- étiquettes de test représentées par un tableau numpy (vecteur) de forme (1, m_test)
    num_iterations -- hyperparamètre représentant le nombre d'itérations pour optimiser les paramètres
    learning_rate -- hyperparamètre représentant le taux d'apprentissage utilisé dans la règle de mise à jour de optimize()
    print_cost -- Mettre à True pour afficher le coût toutes les 100 itérations
    
    Retours :
    d -- dictionnaire contenant des informations sur le modèle.
    """
    # (≈ 1 ligne de code)   
    # initialiser les paramètres avec des zéros
    # et utilisez la fonction "shape" pour obtenir la première dimension de X_train
    # w, b = ...
    
    #(≈ 1 ligne de code)
    # Descente de gradient 
    # params, grads, costs = ...
    
    # Récupérer les paramètres w et b depuis le dictionnaire "params"
    # w = ...
    # b = ...
    
    # Prédire les exemples de l'ensemble de test/train (≈ 2 lignes de code)
    # Y_prediction_test = ...
    # Y_prediction_train = ...
    
    # VOTRE CODE COMMENCE ICI
    
    
    # VOTRE CODE SE TERMINE ICI

    # Afficher les erreurs d'entraînement/test
    if print_cost:
        print("précision train : {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
        print("précision test : {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [ ]:
from public_tests import *

model_test(model)

Si vous réussissez tous les tests, exécutez la cellule suivante pour entraîner votre modèle.

In [ ]:
logistic_regression_model = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations=2000, learning_rate=0.005, print_cost=True)

**Commentaire** : La précision d'entraînement est proche de 100 %. C'est une bonne vérification de cohérence : votre modèle fonctionne et a une capacité suffisante pour s'adapter aux données d'entraînement. La précision de test est de 70 %. Ce n'est en fait pas mal pour ce modèle simple, étant donné le petit jeu de données que nous avons utilisé et le fait que la régression logistique est un classifieur linéaire. 

Aussi, vous voyez que le modèle fait clairement du surapprentissage (overfitting) sur les données d'entraînement. Plus tard dans cette spécialisation, vous apprendrez comment réduire le surapprentissage, par exemple en utilisant la régularisation. En utilisant le code ci-dessous (et en changeant la variable `index`), vous pouvez regarder les prédictions sur les images de l'ensemble de test.

In [ ]:
# Exemple d'une image mal classée.
index = 1
plt.imshow(test_set_x[:, index].reshape((num_px, num_px, 3)))
print ("y = " + str(test_set_y[0,index]) + ", vous avez prédit que c'est une image de \"" + classes[int(logistic_regression_model['Y_prediction_test'][0,index])].decode("utf-8") +  "\".")

Laissez-nous aussi tracer la fonction de coût et les gradients.

In [ ]:
# Tracer la courbe d'apprentissage (avec les coûts)
costs = np.squeeze(logistic_regression_model['costs'])
plt.plot(costs)
plt.ylabel('coût')
plt.xlabel('itérations (par centaines)')
plt.title("Taux d'apprentissage =" + str(logistic_regression_model["learning_rate"]))
plt.show()

**Interprétation** :
Vous pouvez voir le coût diminuer. Cela montre que les paramètres sont en train d'être appris. Cependant, vous voyez que vous pourriez entraîner le modèle encore plus sur l'ensemble d'entraînement. Essayez d'augmenter le nombre d'itérations dans la cellule ci-dessus et relancez les cellules. Vous pourriez voir que la précision de l'ensemble d'entraînement augmente, mais que la précision de l'ensemble de test diminue. C'est ce qu'on appelle le surapprentissage (overfitting). 

<a name='6'></a>
## 6 - Analyse approfondie (optionnel/non noté) ##

Félicitations pour avoir construit votre premier modèle de classification d'images. Analysons-le davantage et examinons les choix possibles pour le taux d'apprentissage $\alpha$. 

#### Choix du taux d'apprentissage ####

**Rappel** :
Pour que la Descente de Gradient fonctionne, vous devez choisir le taux d'apprentissage judicieusement. Le taux d'apprentissage $\alpha$ détermine à quelle vitesse nous mettons à jour les paramètres. Si le taux d'apprentissage est trop grand, nous pouvons "dépasser" la valeur optimale. De même, s'il est trop petit, nous aurons besoin de trop d'itérations pour converger vers les meilleures valeurs. C'est pourquoi il est crucial d'utiliser un taux d'apprentissage bien ajusté.

Comparons la courbe d'apprentissage de notre modèle avec plusieurs choix de taux d'apprentissage. Exécutez la cellule ci-dessous. Cela devrait prendre environ 1 minute. N'hésitez pas à essayer d'autres valeurs que les trois que nous avons initialisées dans la variable `learning_rates` et voyez ce qui se passe.

In [ ]:
learning_rates = [0.01, 0.001, 0.0001]
models = {}

for lr in learning_rates:
    print ("Entraînement d'un modèle avec taux d'apprentissage : " + str(lr))
    models[str(lr)] = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations=1500, learning_rate=lr, print_cost=False)
    print ('\n' + "-------------------------------------------------------" + '\n')

for lr in learning_rates:
    plt.plot(np.squeeze(models[str(lr)]["costs"]), label=str(models[str(lr)]["learning_rate"]))

plt.ylabel('coût')
plt.xlabel('itérations (centaines)')

legend = plt.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
plt.show()

**Interprétation** : 
- Différents taux d'apprentissage donnent différents coûts et donc différents résultats de prédictions.
- Si le taux d'apprentissage est trop grand (0.01), le coût peut osciller de haut en bas. Il peut même diverger (bien que dans cet exemple, l'utilisation de 0.01 finisse quand même par donner une bonne valeur pour le coût). 
- Un coût plus faible ne signifie pas un meilleur modèle. Vous devez vérifier s'il y a possiblement du surapprentissage. Cela se produit lorsque la précision d'entraînement est beaucoup plus élevée que la précision de test.
- En deep learning, nous recommandons généralement de : 
    - Choisir le taux d'apprentissage qui minimise le mieux la fonction de coût.
    - Si votre modèle surapprend, utilisez d'autres techniques pour réduire le surapprentissage. (Nous en parlerons dans des vidéos ultérieures.) 


<a name='7'></a>
## 7 - Test avec votre propre image (optionnel/non noté) ##

Félicitations pour avoir terminé ce devoir. Vous pouvez utiliser votre propre image et voir la sortie de votre modèle. Pour ce faire :
    1. Cliquez sur "File" dans la barre supérieure de ce notebook, puis cliquez sur "Open" pour aller sur votre Coursera Hub.
    2. Ajoutez votre image dans le répertoire de ce Jupyter Notebook, dans le dossier "images"
    3. Changez le nom de votre image dans le code suivant
    4. Exécutez le code et vérifiez si l'algorithme a raison (1 = chat, 0 = non-chat) !

In [ ]:
# changez ceci par le nom de votre fichier image
my_image = "my_image.jpg"   

# Nous prétraitons l'image pour qu'elle s'adapte à votre algorithme.
fname = "images/" + my_image
image = np.array(Image.open(fname).resize((num_px, num_px)))
plt.imshow(image)
image = image / 255.
image = image.reshape((1, num_px * num_px * 3)).T
my_predicted_image = predict(logistic_regression_model["w"], logistic_regression_model["b"], image)

print("y = " + str(np.squeeze(my_predicted_image)) + ", votre algorithme prédit que c'est une image de \"" + classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\".")

<font color='blue'>
    
**Ce qu'il faut retenir de ce devoir :**
1. Le prétraitement du jeu de données est important.
2. Vous avez implémenté chaque fonction séparément : initialize(), propagate(), optimize(). Ensuite, vous avez construit un model().
3. Le réglage du taux d'apprentissage (qui est un exemple d'"hyperparamètre") peut faire une grande différence pour l'algorithme. Vous verrez plus d'exemples de cela plus tard dans ce cours !

Enfin, si vous le souhaitez, nous vous invitons à essayer différentes choses sur ce Notebook. Assurez-vous de soumettre avant d'essayer quoi que ce soit. Une fois que vous avez soumis, les choses avec lesquelles vous pouvez jouer incluent :
    - Jouer avec le taux d'apprentissage et le nombre d'itérations
    - Essayer différentes méthodes d'initialisation et comparer les résultats
    - Tester d'autres prétraitements (centrer les données, ou diviser chaque ligne par son écart type)

Bibliographie :
- http://www.wildml.com/2015/09/implementing-a-neural-network-from-scratch/
- https://stats.stackexchange.com/questions/211436/why-do-we-normalize-images-by-subtracting-the-datasets-image-mean-and-not-the-c